In [3]:
import tensorflow as tf

class SpatialRNNCell(tf.keras.layers.Layer):
    def __init__(self, direction, **kwargs):
        super(SpatialRNNCell, self).__init__(**kwargs)
        self.direction = direction
        # In the literature, alpha is initialized as an identity matrix, but for simplicity, 
        # we're initializing it as a scalar which will have an equivalent effect of scaling the outputs.
        self.alpha = self.add_weight(shape=(1,), initializer="ones", trainable=True)

    def call(self, inputs):
        # Identify the shape of the input feature map.
        feature_map_shape = tf.shape(inputs)
        height = feature_map_shape[1]
        width = feature_map_shape[2]

        # Determine the number of recurrent steps based on the direction.
        num_steps = width if self.direction in ['left', 'right'] else height
        
        # Initialize the output with the input values.
        outputs = inputs
        
        # Perform the recurrent translation.
        for _ in range(num_steps):
            outputs = tf.roll(outputs, shift={'right': (0, 1), 'left': (0, -1),
                                              'up': (-1, 0), 'down': (1, 0)}[self.direction], axis=(1, 2))
            # Apply the alpha weight and ReLU activation at each step.
            outputs = self.alpha * outputs
            outputs = tf.nn.relu(outputs)
        
        return outputs


class AttentionEstimatorNetwork(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionEstimatorNetwork, self).__init__(**kwargs)
        self.conv1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')
        self.conv2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')
        self.conv3 = tf.keras.layers.Conv2D(4, (1, 1), activation=None, padding='same')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        return self.conv3(x)


class DirectionAwareSpatialContextModule(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(DirectionAwareSpatialContextModule, self).__init__(**kwargs)
        self.attention_network = AttentionEstimatorNetwork()
        self.spatial_rnn_cells = {
            direction: SpatialRNNCell(direction=direction)
            for direction in ['right', 'left', 'up', 'down']
        }
        # Layer for the hidden-to-hidden translation
        self.hidden_to_hidden_conv = tf.keras.layers.Conv2D(32, (1, 1), activation='relu')

    def call(self, inputs):
        # Compute attention weights
        W = self.attention_network(inputs)
        W_split = tf.split(W, num_or_size_splits=4, axis=-1)

        # Apply spatial RNN in four directions for the first round
        context_features_1st_round = {
            direction: rnn_cell(inputs)
            for direction, rnn_cell in self.spatial_rnn_cells.items()
        }

        # Multiply first-round context features with attention weights
        weighted_context_features_1st_round = [
            context_features_1st_round[dir] * W_split[i]
            for i, dir in enumerate(['right', 'left', 'up', 'down'])
        ]

        # Concatenate features and apply 1x1 convolution to reduce dimensionality
        concatenated_features_1st_round = tf.concat(weighted_context_features_1st_round, axis=-1)
        reduced_features_1st_round = self.hidden_to_hidden_conv(concatenated_features_1st_round)

        # Apply spatial RNN in four directions for the second round
        context_features_2nd_round = {
            direction: rnn_cell(reduced_features_1st_round)
            for direction, rnn_cell in self.spatial_rnn_cells.items()
        }

        # Multiply second-round context features with the SAME attention weights
        weighted_context_features_2nd_round = [
            context_features_2nd_round[dir] * W_split[i]
            for i, dir in enumerate(['right', 'left', 'up', 'down'])
        ]

        # Concatenate features from the second round
        concatenated_features_2nd_round = tf.concat(weighted_context_features_2nd_round, axis=-1)

        # Final 1x1 convolution followed by ReLU as per the literature
        output_dsc_features = self.hidden_to_hidden_conv(concatenated_features_2nd_round)

        return output_dsc_features


In [4]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Conv2D, Input, UpSampling2D, concatenate
from tensorflow.keras.models import Model

# Assume that the DirectionAwareSpatialContextModule and other necessary classes from previous steps are already implemented.

# Initialize VGG with weights pre-trained on ImageNet
vgg = VGG16(include_top=False, weights='imagenet', input_tensor=Input(shape=(224, 224, 3)))

# Apply DSC modules to each layer's output except the first
feature_maps = [vgg.get_layer(name).output for name in vgg.layers]
dsc_feature_maps = [DirectionAwareSpatialContextModule()(feature) for feature in feature_maps[1:]]

# Initialize the output layer of the network for shadow detection
output_layers = []

for dsc_feature in dsc_feature_maps:
    # 1x1 convolution and upsampling to match the original image size, if needed
    upsampled = UpSampling2D(size=(2, 2))(dsc_feature)  # Adjust the size accordingly
    output_layer = Conv2D(1, (1, 1), activation='sigmoid')(upsampled)
    output_layers.append(output_layer)

# Concatenate all output layers and apply a final 1x1 convolution
concatenated_outputs = concatenate(output_layers)
final_output = Conv2D(1, (1, 1), activation='sigmoid', name='fusion_layer')(concatenated_outputs)

# Define the overall model
model = Model(inputs=vgg.input, outputs=final_output)

# Define the weighted cross-entropy loss function
def weighted_cross_entropy_loss(y_true, y_pred, Np, Nn, TP, TN):
    Ldi = -(Nn / (Np + Nn)) * y_true * tf.math.log(y_pred) - (Np / (Np + Nn)) * (1 - y_true) * tf.math.log(1 - y_pred)
    Lai = -(1 - TP / Np) * y_true * tf.math.log(y_pred) - (1 - TN / Nn) * (1 - y_true) * tf.math.log(1 - y_pred)
    return Ldi + Lai

# Define a custom training loop or use `model.compile` and `model.fit` with the loss function
model.compile(optimizer='sgd', loss=weighted_cross_entropy_loss, metrics=['accuracy'])

# Custom training loop (simplified and conceptual)
for image, mask in dataset:
    with tf.GradientTape() as tape:
        predictions = model(image)
        # Calculate loss for each layer and sum them
        loss = sum([weighted_cross_entropy_loss(mask, pred, Np, Nn, TP, TN) for pred in predictions])
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

# Continue with the training procedure as defined in the literature


ValueError: No such layer: <InputLayer name=input_layer, built=True>. Existing layers are: ['input_layer', 'block1_conv1', 'block1_conv2', 'block1_pool', 'block2_conv1', 'block2_conv2', 'block2_pool', 'block3_conv1', 'block3_conv2', 'block3_conv3', 'block3_pool', 'block4_conv1', 'block4_conv2', 'block4_conv3', 'block4_pool', 'block5_conv1', 'block5_conv2', 'block5_conv3', 'block5_pool'].